In [23]:
import pandas as pd
import string
import os
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sukku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sukku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("combined_data.csv")

In [7]:
df.sample(5)

,label,text
53353,0,"we agree\n"" eileen ponton "" on 03 / 20 / 2001 ..."
33077,0,kal - please ensure you send this information ...
19615,1,http temperaturegave hk viagraescapenumber pil...
67739,0,make your synth emulation table driven that wa...
18549,1,take advantage of mycanadianpharmacy's special...


In [4]:
stop_words = set(stopwords.words('english'))

In [37]:
ps = PorterStemmer()
def text_transform(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    #text = text.translate(str.maketrans('', '', string.punctuation))
    text  = str.split(text)
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if word not in stop_words]
    # text = [ps.stem(word) for word in text]   # stem each word
    return text

text_transform("Hello this name is Ahmed n Raza")

['hello', 'ahmed', 'n', 'raza']

In [38]:
df['text'] = df['text'].apply(text_transform)

In [40]:
df.head()

,label,text
0,1,"[ounce, feather, bowl, hummingbird, opec, mome..."
1,1,"[wulvob, medircations, online, qnb, ikud, viag..."
2,0,"[computer, connection, cnn, com, wednesday, es..."
3,1,"[university, degree, obtain, prosperous, futur..."
4,0,"[thanks, answers, guys, know, checked, rsync, ..."


In [41]:
s = df['text'].to_list()

model = Word2Vec()

In [42]:
model.build_vocab(s)

In [43]:
model.train(s, total_examples=model.corpus_count, epochs=model.epochs)

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


(61316006, 70399625)

In [47]:
import numpy as np
# Cell 9 - Create document embeddings for classification
def get_doc_vector(tokens, model, vector_size=100):
    """Average all word vectors in a document to get a single document vector"""
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

X = np.array([get_doc_vector(tokens, model) for tokens in df['text']])
y = df['label'].values  # Make sure your label column is named 'label'

print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")

Feature matrix shape: (83448, 100)
Labels shape: (83448,)


In [48]:
# Cell 10 - Train a classifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9845
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      7938
           1       0.98      0.99      0.99      8752

    accuracy                           0.98     16690
   macro avg       0.98      0.98      0.98     16690
weighted avg       0.98      0.98      0.98     16690



In [66]:
# Cell - Predict on new email
sample_email = "Can you review my code before submission?"
sample_tokens = text_transform(sample_email)
sample_vector = get_doc_vector(sample_tokens, model, vector_size=100).reshape(1, -1)
prediction = clf.predict(sample_vector)

print("Spam" if prediction[0] == 1 else "Not Spam")

Not Spam
